In [1]:
import pandas as pd
#import janitor as jn

In [2]:
pd.set_option('max_columns', None)

## Charger les données

In [3]:
data = pd.read_pickle("../../data/processed/maison.pkl")
#data.to_pickle("../../data/processed/maison.pkl")

In [4]:
# Afficher la table initiale
print(data.info(null_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315650 entries, 0 to 315649
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   valeur_fonciere            315650 non-null  float64 
 1   adresse_code_voie          315650 non-null  category
 2   code_postal                315650 non-null  category
 3   code_commune               315650 non-null  category
 4   code_departement           315650 non-null  category
 5   surface_reelle_bati        315650 non-null  float64 
 6   nombre_pieces_principales  315650 non-null  float64 
 7   code_nature_culture        315650 non-null  category
 8   surface_terrain            315650 non-null  float64 
dtypes: category(5), float64(4)
memory usage: 14.1 MB
None


## Traitement des données

In [5]:
num_features = data.select_dtypes("float").columns
categ_features = data.select_dtypes("category").columns

In [43]:
data[categ_features] = data[categ_features].astype("str")
data[num_features] = data[num_features].astype("float32")

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315650 entries, 0 to 315649
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   valeur_fonciere            315650 non-null  float32
 1   adresse_code_voie          315650 non-null  object 
 2   code_postal                315650 non-null  object 
 3   code_commune               315650 non-null  object 
 4   code_departement           315650 non-null  object 
 5   surface_reelle_bati        315650 non-null  float32
 6   nombre_pieces_principales  315650 non-null  float32
 7   code_nature_culture        315650 non-null  object 
 8   surface_terrain            315650 non-null  float32
dtypes: float32(4), object(5)
memory usage: 16.9+ MB


In [45]:
#data["code_postal"] = data["code_postal"].astype("str")
cat_f_to_keep = ["code_postal"]
cat_f_to_ignore = categ_features[categ_features!=cat_f_to_keep[0]]

target = 'valeur_fonciere'

num_f_to_keep = list(num_features[num_features!=target])

In [51]:
sub_pc = (data["code_postal"].value_counts().cumsum()<120000)
list_sub_pc = list(sub_pc.index[sub_pc])
df = data[data["code_postal"].isin(list_sub_pc)]

df = df[(df["valeur_fonciere"]>30000) & (df["valeur_fonciere"]<300000)]
print(data.shape, df.shape)

(315650, 9) (6628, 9)


In [16]:
df = data[(data["valeur_fonciere"]>30000) & (data["valeur_fonciere"]<300000)]
print(data.shape, df.shape)

(315650, 9) (243287, 9)


## Pycaret - Regression

PROB : pour la scission des données, faire proportionnel dans chaque classe. Mais il y a peut-etre des classes manquantes dans les données d'entrainement que l'on a seulement dans l'ensemble de test.

In [13]:
from pycaret.regression import *

In [52]:
exp_reg101 = setup(data = df, target = target, ignore_features=cat_f_to_ignore, categorical_features=cat_f_to_keep,
                   numeric_features=num_f_to_keep,
                   remove_outliers=True, #remove_multicollinearity = True,# multicollinearity_threshold = 0.6,
                   #combine_rare_levels = True, rare_level_threshold=0.2,
                   polynomial_features = True,
                   trigonometry_features = True,
                   feature_interaction=True,
                   feature_ratio=True,
                   create_clusters=True,
                   use_gpu=True,
                   normalize = True, normalize_method="robust", transformation = True, transform_target = True, verbose=True, session_id=123)
# normalize_method='robust'

 
Setup Succesfully Completed.


,Description,Value
0,session_id,123
1,Transform Target,True
2,Transform Target Method,box-cox
3,Original Data,"(6628, 9)"
4,Missing Values,False
5,Numeric Features,5
6,Categorical Features,3
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [ ]:
# exp_reg101 = setup(data = data, target = 'valeur_fonciere', remove_outliers=True,
#                    ignore_features=cat_features, create_clusters=True, polynomial_features=True, feature_ratio=True, 
#                    profile=True, session_id=123) 
# exp_reg101 = setup(data = data, target = target, remove_outliers=True, categorical_features=cat_f_tokeep,
#                    numeric_features=num_f_tokeep, ignore_features=cat_f_toignore,
#                    session_id=123) 

In [53]:
compare_models(exclude=["catboost", "svm", "et", "tr", "rf", "ransac", "gbr","ada"], round=4, fold=5, n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Lasso Least Angle Regression,8422.7936,96961679.8430,9846.4943,-0.0035,0.2069,0.1825,0.0500
1,Lasso Regression,8444.3850,97457360.0366,9871.5929,-0.0086,0.2074,0.1830,0.4068
2,Elastic Net,8452.0132,97704232.6577,9884.0324,-0.0111,0.2077,0.1831,0.5171
3,Bayesian Ridge,8459.0942,98172685.8848,9907.3612,-0.0159,0.2093,0.1833,0.3416
4,Huber Regressor,8464.0308,98232497.6927,9910.3991,-0.0165,0.2086,0.1836,2.2889
5,Orthogonal Matching Pursuit,8424.4908,99805641.2097,9989.2870,-0.0328,0.2091,0.1822,0.0582
6,Ridge Regression,8486.7595,104053050.1590,10196.8749,-0.0762,0.3349,0.1828,0.0518
7,Light Gradient Boosting Machine,8545.3583,104208391.6485,10206.7599,-0.0783,0.2128,0.1842,1.2291
8,Extreme Gradient Boosting,8713.5539,109156928.0000,10444.8441,-0.1291,0.2173,0.1873,3.7371
9,K Neighbors Regressor,8859.3119,113846880.0000,10668.3701,-0.1780,0.2216,0.1906,0.2222


[LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
           fit_path=True, jitter=None, max_iter=500, normalize=True,
           positive=False, precompute='auto', random_state=None, verbose=False),
 Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
       normalize=False, positive=False, precompute=False, random_state=123,
       selection='cyclic', tol=0.0001, warm_start=False),
 ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
            max_iter=1000, normalize=False, positive=False, precompute=False,
            random_state=123, selection='cyclic', tol=0.0001, warm_start=False)]

In [57]:
llar = create_model("llar")
mlp = create_model("mlp")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,28790.5509,51051448496.3527,225945.6760,-577.8834,1.3291,0.5763
1,24525.3201,23802114591.1408,154279.3395,-230.3811,1.8211,0.5140
2,13056.5051,1847114555.4504,42978.0706,-18.1435,1.5691,0.2783
3,14488.7416,2310469065.3199,48067.3389,-23.1905,1.8786,0.2918
4,29347.2733,116955923230.6604,341988.1917,-1095.9750,2.0310,0.6888
5,15094.1894,6642934263.6349,81504.1978,-71.5586,1.5662,0.3188
6,52566.5892,123514891833.6324,351446.8549,-1321.3703,1.9333,1.0386
7,20772.7456,47320720978.9233,217533.2641,-499.8465,1.4043,0.3799
8,11624.5655,313409838.9595,17703.3850,-2.2819,1.5764,0.2467
9,27419.6840,73375283198.0676,270878.7242,-727.0600,1.2141,0.5456


In [61]:
tune_llar = tune_model(llar, n_iter=1000)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,8001.7424,88000166.7001,9380.8404,0.0021,0.1951,0.1707
1,8733.6652,102997941.6577,10148.7902,-0.0012,0.2134,0.1895
2,8449.2756,95859051.1800,9790.7636,0.0065,0.2051,0.1823
3,8340.7053,95697950.4711,9782.5329,-0.0020,0.2067,0.1827
4,8950.9794,105965543.1154,10293.9566,0.0061,0.2161,0.1939
5,8166.4693,91006447.9247,9539.7300,0.0060,0.1990,0.1751
6,8187.8982,93651666.8560,9677.3791,-0.0026,0.2056,0.1804
7,8347.2690,94299387.5692,9710.7872,0.0019,0.2015,0.1775
8,8240.3795,96681350.5259,9832.6675,-0.0124,0.2103,0.1834
9,8560.0327,100049465.4886,10002.4730,0.0073,0.2097,0.1843


In [64]:
evaluate_model(tune_llar)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
tune_mlp = tune_model(mlp, n_iter=100)

In [54]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model.Lasso,True
ridge,Ridge Regression,sklearn.linear_model.Ridge,True
en,Elastic Net,sklearn.linear_model.ElasticNet,True
lar,Least Angle Regression,sklearn.linear_model.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model.OMP,True
br,Bayesian Ridge,sklearn.linear_model.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model.ARDRegression,False


## Restes

In [ ]:
interpret_model(xg_reg)

In [ ]:
xg_reg = automl(use_holdout=False)
xg_reg

In [ ]:
plot_model(xg_reg, plot = 'residuals')

In [ ]:
pull()

In [ ]:
xg_reg = create_model("xgboost")
tune_xg_reg = tune_model(xg_reg, n_iter=100, fold=10, choose_better=True)
print(tune_xg_reg)

In [ ]:
tune_xg_reg2 = tune_model(tune_xg_reg2, n_iter=10, fold=10, choose_better=True)
print(tune_xg_reg)

In [ ]:
print(tune_xg_reg2)

In [ ]:
finalize_model(xg_reg)

In [ ]:
save_model(xg_reg,'../../models/final_xg_reg_for_streamlit_test')

In [ ]:
predict_model(xg_reg)

In [ ]:
interpret_model(xg_reg)

In [ ]:
rf_reg = create_model("rf")
tune_rf_reg = tune_model(rf_reg, n_iter=100, fold=10, choose_better=True)
print(tune_rf_reg)

In [ ]:
compare_models(whitelist=["catboost", "svm"], fold=5, n_select=2)

In [ ]:
compare_models(blacklist=["catboost"], fold=5, n_select=1)

In [ ]:
catboost_reg = create_model("catboost")

In [ ]:
tune_huber_reg = tune_model(huber_reg, n_iter=100, fold=10, choose_better=True)
print(tune_huber_reg)

In [ ]:
#evaluate_model(tune_huber_reg)

In [ ]:
interpret_model(tune_huber_reg)

In [ ]:
exp_reg102 = setup(data = data, target = 'valeur_fonciere', remove_outliers=True,
                   ignore_features=cat_features, polynomial_features=True,
                   session_id=124) 

In [ ]:
compare_models(blacklist=["catboost","svm"], fold=5, n_select=1)

In [ ]:
lr_reg = create_model("lr")

In [ ]:
tune_lr_reg = tune_model(lr_reg, n_iter=100, fold=10, choose_better=True)
print(tune_lr_reg)

In [ ]:
interpret_model(tune_lr_reg)

In [ ]:
blend_all = blend_models()

In [ ]:
data["code_postal"] = data["code_postal"].apply(lambda x: f"{x}")

In [ ]:
data["code_postal"]

In [ ]:
pd.Series.apply()